In [1]:

# coding: utf-8

# In[1]:
import matplotlib as mpl   
mpl.use('Agg')

import sys
sys.path.append('lib')
# lib contains util, functions, models, image, normalizers, extract_xml, functools

from openslide import OpenSlide
from os import listdir
from os.path import join, isfile, exists, splitext
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure
import time
from skimage.viewer import ImageViewer
import tensorflow as tf
import keras
import warnings
warnings.filterwarnings("ignore")   
from scipy import interpolate

from util import otsu_thresholding, center_of_slide_level,connected_component_image
from scipy import ndimage
from functions import *
#from models import getModel


# In[2]:


from keras.preprocessing import image
from keras.applications import resnet50
import keras.applications.inception_v3
import sys
sys.path.insert(0, '/mnt/nas2/results/Models')
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from resnet101 import *
from googlenet import *
from keras.preprocessing import image
import tensorflow as tf

def getModel(net_settings, num_classes=1, imagenet_weights_path=""):
    '''
        Should be modified with model type as input and returns the desired model
    '''
    if net_settings['model_type'] == 'resnet':
        base_model = resnet50.ResNet50(include_top=True, weights='imagenet')
        finetuning = Dense(1, activation='sigmoid', name='predictions')(base_model.layers[-2].output)
        model = Model(input=base_model.input, output=finetuning)
        model.compile(loss=net_settings['loss'],
        optimizer=optimizers.SGD(lr=net_settings['lr'],  momentum=0.9, decay=1e-6, nesterov=True),
        metrics=['accuracy'])
        return model
    elif net_settings['model_type'] == 'resnet101':
        model = resnet101_model(224, 224, 3, 1,imagenet_weights_path)
        return model
    elif net_settings['model_type']=='inceptionv3':
        base_model = keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet')
        finetuning = Dense(1, activation='sigmoid', name='predictions')(base_model.layers[-2].output)
        model = Model(input=base_model.input, output=finetuning)
        model.compile(loss=net_settings['loss'],
                      optimizer=optimizers.SGD(lr=net_settings['lr'],  momentum=0.9, decay=1e-6, nesterov=True),
                      metrics=['accuracy'])
        return model
    elif net_settings['model_type'] == 'googlenet':
        model = check_print()
        return model
    else:
        print '[models] Ugggh. Not ready for this yet.'
        exit(0)
        return None


# In[3]:


import setproctitle
EXPERIMENT_TYPE = 'distributed_inference'
# SET PROCESS TITLE
setproctitle.setproctitle('UC1_{}_{}'.format(EXPERIMENT_TYPE, 'mara'))
CONFIG_FILE='doc/config.cfg'


# In[4]:


config = ConfigParser.RawConfigParser(allow_no_value = True)
config.read(CONFIG_FILE)

input_file = config.get("input", "file_name")
xml_source = config.get("settings", "xml_source_fld")
data_folder = config.get("settings", "source_fld")
imagenet_weights_path = config.get("input", "imagenet_weights")
model_weights=config.get("input", "model_weights")
interpret_=config.get("input", "interpret")
n_samples_max=config.get("input", "n_samples")


# In[5]:


os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.visible_device_list = sys.argv[2] 
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

folder='./model_weights/'
#0609-1648/'


settings = parseTrainingOptions(CONFIG_FILE)
print settings

model=getModel(settings, imagenet_weights_path=imagenet_weights_path)
model.load_weights(model_weights)


# In[6]:


if interpret_:
    import interpret
    reload(interpret)
    from interpret import *
    print "interpreting network"
    res_folder = 'results/'
    new_folder = res_folder + 'interpretability/'
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    new_folder = res_folder + 'interpretability/{}'.format(input_file)
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    n_samples=0
    input_size=(224,224)


# In[7]:


pwd=""
from os.path import join,isfile, exists, splitext
def get_folder(file_name, path=""):
    for fold in os.listdir(path)[:4]:
        m = file_name+".tif"
        if m in os.listdir(path+fold):
            return fold
    return path
def preprocess_test_data(slide_path, slide_level=7, patch_size=224, verbose=1):
    rgb_im, slide = load_slide(slide_path, slide_level=slide_level)
    otsu_im = get_otsu_im(rgb_im, verbose = 0)
    return slide, rgb_im, otsu_im

def apply_morph(otsu_im):
    im_gray_ostu = otsu_im
    kernel = np.ones((2,2),np.uint8)
    kernel_1 = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(im_gray_ostu,cv2.MORPH_OPEN,kernel)
    opening_1 = cv2.morphologyEx(im_gray_ostu,cv2.MORPH_OPEN,kernel_1)
    closing = cv2.morphologyEx(im_gray_ostu,cv2.MORPH_CLOSE,kernel)
    opening_1= np.abs(255-opening_1)
    return opening_1

def preprocess_test_data(slide_path, slide_level=7, patch_size=224, verbose=1):
    rgb_im, slide = load_slide(slide_path, slide_level=slide_level)
    otsu_im = get_otsu_im(rgb_im, verbose = 0)
    return slide, rgb_im, otsu_im

def apply_morph(otsu_im):
    im_gray_ostu = otsu_im
    kernel = np.ones((2,2),np.uint8)
    kernel_1 = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(im_gray_ostu,cv2.MORPH_OPEN,kernel)
    opening_1 = cv2.morphologyEx(im_gray_ostu,cv2.MORPH_OPEN,kernel_1)
    closing = cv2.morphologyEx(im_gray_ostu,cv2.MORPH_CLOSE,kernel)
    opening_1= np.abs(255-opening_1)
    return opening_1

training_slide=True
file_name = input_file
print file_name
xml_path=xml_source+file_name+'.xml'
folder = get_folder(file_name,path=data_folder)
print folder
filename =data_folder+folder+"/"+file_name+'.tif' 
slide_path = join(pwd,filename)
print "file name : "+slide_path+"\n"
if isfile(slide_path):
    """is it file? """
    slide=OpenSlide(slide_path)
elif exists(slide_path):
    """ dose it exist? """
    print "slide_path :" + slide_path + " is not a readable file \n"
else:
    """ it is not a file and doesn't exist"""
    print "file dosen't exist in this path :"  + slide_path+"\n"

slide_w, slide_h = slide.dimensions
print "Whole Slide dimensions (with, heigth):{0}\n".format(slide.dimensions)
#Slide.level_dimensions
slide_level = 7
s_level_w, s_level_h = slide.level_dimensions[slide_level]
print "slide.level_count-1 dimensions (width,heigth):{0}\n".format(slide.level_dimensions[slide_level])
slide, rgb_im, otsu_im = preprocess_test_data(slide_path, slide_level=7, patch_size=224, verbose=1)

if not training_slide:
    slide, rgb_im, otsu_im = preprocess_test_data(slide_path, slide_level=7, patch_size=224, verbose=1)

    ## to be continued....
else:
    slide, annotations_mask, rgb_im, im_contour = preprocess(slide_path, xml_path, slide_level=slide_level)


# In[8]:


dmodels={}
import subprocess as sp
def get_gpu_memory():
    _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
    ACCEPTABLE_AVAILABLE_MEMORY = 1024
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    print(memory_free_values)
    return memory_free_values

free_gpu_memory_ = get_gpu_memory()
MEMO_REQUIREMENT = 3000.
n_models = int(free_gpu_memory_[0] / MEMO_REQUIREMENT)
# Distribute inference : GPU Parallelism
print "Distributing inference over {} model copies".format(n_models)
for i in range(0, n_models):
    try: 
        print "Instantiating model n. ", i
        nmodel=getModel(settings, imagenet_weights_path=imagenet_weights_path)
        print "Loading weights..."
        nmodel.load_weights(model_weights)
        print "Adding model to available models."
        dmodels[i]=nmodel
        i+=1
    except:
        print "Something went wrong. Check the memory status. Otherwise failed loading model weights"
        break


# In[9]:


import multiprocessing
opening_1 = apply_morph(otsu_im)
plt.rcParams['figure.figsize']=(5,5)
y_low, x = np.unique(np.where(opening_1>0)[0]), np.unique(np.where(opening_1>0)[1]) 
patch_size=224
patches=[]
flag=False
dim_x, dim_y=slide.level_dimensions[slide_level]
print 'Heatmap dimensions: ', dim_x, dim_y


# In[10]:


heatmap=np.zeros((slide.level_dimensions[slide_level][1], slide.level_dimensions[slide_level][0]))
seen=np.zeros((slide.level_dimensions[slide_level][1], slide.level_dimensions[slide_level][0]))        
resolution=2
mesh = np.meshgrid(np.arange(0, slide.level_dimensions[slide_level][0], 2),np.arange(0, slide.level_dimensions[slide_level][1], 2))
positions = np.vstack(map(np.ravel, mesh)).T
final_p=[]
to_interpret=[]
to_interpret_loc=[]
for p in positions:
    x,y=p[0],p[1]
    if np.sum(opening_1[y-(resolution/2):y+(resolution/2), x-(resolution/2):x+(resolution/2)])>0:
        if y>50 and y<(dim_y-50) and x>50 and x<(dim_x-50):
            final_p.append(p)


# In[ ]:

def worker(slide, locations_vector, locations_index, data_batch, data_locations, batch_size=32):
    """worker function for multiprocessing"""
    batch=[]
    batch_locations = locations_vector[locations_index.value:locations_index.value+batch_size]
    for l in batch_locations:
        #l[0] is x, l[1] is y
        patch=np.asarray(slide.read_region((l[0]*128,l[1]*128),0,(224,224)))[...,:3]
        batch.append(np.asarray(patch)[...,:3])
        print l[0], l[1]
        #Image.fromarray(patch).save('prova_batch/{}-{}.png'.format(l[1],l[0]))
    data_batch[0]=batch
    data_locations[0]=batch_locations
    with locations_index.get_lock():
        locations_index.value +=batch_size
    return

start_time = time.time()
locations_index = multiprocessing.Value("i", 0)
manager = multiprocessing.Manager()
batches = {}
locations = {}
for b in range(n_models):
    batches[b]=manager.dict()
    locations[b]=manager.dict()
#batches=manager.dict()
batch_size=32
n_batches_=0
while locations_index.value < len(final_p):
    jobs = []
    for m in range(n_models):
        p = multiprocessing.Process(target=worker, 
                                    args=(slide, 
                                          final_p, 
                                          locations_index, 
                                          batches[m], 
                                          locations[m]))
        jobs.append(p)
        p.start() 
        p.join()
        predictions=dmodels[m].predict(np.reshape(batches[m][0],(len(batches[m][0]),224,224,3)))
        n_batches_+=1
        for p in range(len(predictions)):
            x_b, y_b=locations[m][0][p][0], locations[m][0][p][1]
            heatmap[y_b, x_b]=predictions[p][0]
            if interpret and predictions[p][0]>0.82 and n_samples<int(n_samples_max) and n_batches_>200:
                print n_samples, n_samples_max, predictions[p][0], n_samples<int(n_samples_max)
                pred_layer = dmodels[m].layers[-1].name
                inputs = np.expand_dims(batches[m][0][p], axis=0)
                if settings['model_type']=='resnet101':
                    conv_layer='res5c_relu'
                elif settings['model_type']=='inceptionv3':
                    conv_layer='mixed10'
                else:
                    conv_layer=model.layers[-3].name
                cam_=cam(model, inputs, conv_layer, input_size)
                #plt.figure()
                #plt.imshow(cam_)
                plt.rcParams['figure.figsize']=(5,5)
                #plt.savefig('{}/{}_{}'.format(new_folder,x_b,y_b))
                plt.figure()
                plt.title('(x, y) coordinates: ({}, {}) prediction: {}'.format(x_b, y_b, predictions[p][0]))
                plt.imshow(np.uint8(batches[m][0][p]))
                plt.imshow(cam_, alpha=.6, cmap='jet')
                plt.savefig('{}/{}_{}_overlay'.format(new_folder,x_b,y_b))
                n_samples+=1
            seen[y_b,x_b]=1

            
            
end_time = time.time()

points = np.asarray(seen.nonzero()).T
values = heatmap[heatmap.nonzero()]

grid_x, grid_y = np.mgrid[0:slide.level_dimensions[slide_level][1]:1, 
                          0:slide.level_dimensions[slide_level][0]:1]

# In[ ]:

# In[ ]:

try:
    interpolated_heatmap = interpolate.griddata(points, values,
                                      (grid_x, grid_y), 
                                        fill_value=1.
                                       )
except:
    interpolated_heatmap=heatmap
plt.rcParams['figure.figsize']=(25,25)
plt.figure()
plt.imshow(im_contour)
plt.imshow(interpolated_heatmap, alpha=0.5, cmap='jet')
plt.savefig('results/{}_interpolated'.format(file_name))
plt.figure()
plt.imshow(seen)
plt.savefig('results/{}_seen'.format(file_name))
print 'Number of patches analysed: ', np.sum(seen)
print 'Elapsed time: ', end_time-start_time
plt.rcParams['figure.figsize']=(25,25)
plt.figure()
plt.imshow(im_contour)
plt.imshow(heatmap, cmap="jet", alpha=0.5)
plt.savefig('results/{}'.format(file_name))
f=open('results/{}_log.txt'.format(file_name),'w')
f.write('Number of patches analysed: {}\n'.format(np.sum(seen)))
f.write('Elapsed time: {} s'.format(end_time-start_time))
f.close()


/home/mara/venv/local/lib/python2.7/site-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')
/home/mara/venv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


{'loss': 'binary_crossentropy', 'verbose': 1, 'decay': 1e-06, 'nesterov': 'True', 'activation': 'sigmoid', 'batch_size': 32, 'epochs': 15, 'lr': 0.0001, 'model_type': 'inceptionv3', 'momentum': 0.9}
interpreting network
patient_015_node_1
centre_0
file name : /mnt/nas4/datasets/ToReadme/CAMELYON17/working_subset/centre_0/patient_015_node_1.tif

Whole Slide dimensions (with, heigth):(94968, 210579)

slide.level_count-1 dimensions (width,heigth):(741, 1645)

[ 3.  2.  2. ...  2.  3. 18.]
Otsu threshold:  185.314453125
[functions][data_preprocessing] NEW PREPROCESS FUNCTION
[6550]
Distributing inference over 2 model copies
Instantiating model n.  0
Loading weights...
Adding model to available models.
Instantiating model n.  1
Loading weights...
Adding model to available models.
Heatmap dimensions:  741 1645
194 776
196 776
198 776
200 776
212 776
214 776
216 776
222 776
224 776
226 776
228 776
230 776
232 776
234 776
236 776
238 776
240 776
242 776
244 776
192 778
194 778
196 778
198 778


178 872
180 872
182 872
184 872
186 872
188 872
190 872
192 872
194 872
196 872
198 872
200 872
202 872
204 872
206 872
208 872
210 872
212 872
214 872
216 872
218 872
220 872
222 872
224 872
226 872
228 872
230 872
336 872
338 872
340 872
342 872
370 872
396 872
398 872
400 872
440 872
442 872
444 872
446 872
466 872
570 872
572 872
574 872
576 872
578 872
580 872
586 872
588 872
590 872
606 872
608 872
614 872
616 872
618 872
620 872
622 872
624 872
626 872
628 872
630 872
632 872
336 874
338 874
340 874
342 874
396 874
398 874
422 874
446 874
472 874
474 874
476 874
478 874
570 874
572 874
574 874
576 874
578 874
580 874
582 874
584 874
586 874
588 874
590 874
592 874
594 874
596 874
598 874
600 874
602 874
604 874
606 874
608 874
610 874
612 874
614 874
616 874
618 874
620 874
622 874
624 874
626 874
628 874
630 874
632 874
336 876
338 876
340 876
342 876
366 876
392 876
394 876
396 876
398 876
422 876
476 876
478 876
480 876
482 876
484 876
504 876
570 876
572 876
574 876
576 876


408 924
410 924
412 924
414 924
416 924
418 924
420 924
422 924
424 924
426 924
428 924
430 924
432 924
434 924
436 924
438 924
440 924
442 924
444 924
446 924
448 924
450 924
452 924
454 924
456 924
458 924
460 924
462 924
464 924
466 924
468 924
470 924
472 924
474 924
476 924
478 924
480 924
482 924
484 924
486 924
488 924
490 924
544 924
546 924
548 924
550 924
552 924
554 924
556 924
558 924
560 924
580 924
582 924
584 924
602 924
604 924
606 924
652 924
654 924
336 926
338 926
340 926
342 926
402 926
404 926
406 926
408 926
410 926
412 926
414 926
416 926
418 926
420 926
422 926
424 926
426 926
428 926
430 926
432 926
434 926
436 926
438 926
440 926
442 926
444 926
446 926
448 926
450 926
452 926
454 926
456 926
458 926
460 926
462 926
464 926
466 926
468 926
470 926
472 926
474 926
476 926
478 926
480 926
482 926
484 926
486 926
488 926
490 926
492 926
494 926
496 926
544 926
546 926
548 926
550 926
552 926
554 926
556 926
558 926
560 926
580 926
582 926
584 926
602 926
604 926


478 950
480 950
482 950
484 950
486 950
488 950
490 950
492 950
494 950
496 950
498 950
500 950
502 950
504 950
506 950
508 950
538 950
540 950
574 950
576 950
622 950
652 950
654 950
656 950
216 952
218 952
220 952
222 952
224 952
226 952
228 952
230 952
232 952
234 952
236 952
238 952
240 952
242 952
244 952
246 952
248 952
250 952
252 952
254 952
256 952
258 952
260 952
272 952
274 952
276 952
278 952
280 952
326 952
328 952
370 952
372 952
374 952
400 952
402 952
404 952
406 952
408 952
410 952
412 952
414 952
416 952
418 952
420 952
422 952
424 952
426 952
428 952
430 952
432 952
434 952
436 952
438 952
440 952
442 952
444 952
446 952
448 952
450 952
452 952
454 952
456 952
458 952
460 952
462 952
464 952
466 952
468 952
470 952
472 952
474 952
476 952
478 952
480 952
482 952
484 952
486 952
488 952
490 952
492 952
494 952
496 952
498 952
500 952
502 952
504 952
506 952
508 952
540 952
542 952
544 952
546 952
548 952
620 952
622 952
624 952
652 952
654 952
656 952
216 954
218 954


...

276 1120
278 1120
280 1120
282 1120
284 1120
286 1120
288 1120
290 1120
292 1120
294 1120
296 1120
298 1120
300 1120
302 1120
304 1120
306 1120
308 1120
310 1120
312 1120
314 1120
316 1120
318 1120
320 1120
322 1120
324 1120
326 1120
328 1120
330 1120
332 1120
334 1120
336 1120
338 1120
340 1120
342 1120
344 1120
346 1120
348 1120
350 1120
352 1120
354 1120
356 1120
358 1120
360 1120
362 1120
364 1120
366 1120
368 1120
370 1120
372 1120
374 1120
376 1120
378 1120
380 1120
382 1120
384 1120
386 1120
388 1120
390 1120
392 1120
394 1120
396 1120
398 1120
400 1120
402 1120
404 1120
406 1120
416 1120
448 1120
112 1122
114 1122
116 1122
142 1122
144 1122
146 1122
148 1122
150 1122
152 1122
154 1122
156 1122
158 1122
160 1122
162 1122
164 1122
166 1122
168 1122
170 1122
172 1122
174 1122
176 1122
178 1122
180 1122
182 1122
194 1122
196 1122
198 1122
200 1122
202 1122
204 1122
206 1122
208 1122
210 1122
212 1122
214 1122
220 1122
222 1122
224 1122
226 1122
228 1122
230 1122
232 1122
234 1122
2

258 1134
260 1134
262 1134
264 1134
266 1134
268 1134
270 1134
272 1134
274 1134
276 1134
278 1134
280 1134
282 1134
284 1134
286 1134
288 1134
290 1134
292 1134
294 1134
296 1134
298 1134
300 1134
302 1134
304 1134
306 1134
308 1134
310 1134
312 1134
314 1134
316 1134
318 1134
320 1134
322 1134
324 1134
326 1134
328 1134
330 1134
332 1134
334 1134
336 1134
338 1134
340 1134
342 1134
344 1134
346 1134
348 1134
350 1134
352 1134
354 1134
356 1134
358 1134
360 1134
362 1134
364 1134
366 1134
368 1134
370 1134
372 1134
374 1134
376 1134
378 1134
380 1134
382 1134
384 1134
386 1134
436 1134
438 1134
114 1136
116 1136
148 1136
150 1136
152 1136
154 1136
156 1136
158 1136
160 1136
162 1136
164 1136
166 1136
168 1136
170 1136
172 1136
174 1136
176 1136
178 1136
180 1136
182 1136
184 1136
186 1136
188 1136
190 1136
192 1136
194 1136
196 1136
198 1136
200 1136
202 1136
204 1136
206 1136
208 1136
210 1136
212 1136
214 1136
216 1136
218 1136
220 1136
222 1136
224 1136
226 1136
228 1136
230 1136
2

240 1148
242 1148
244 1148
246 1148
248 1148
250 1148
252 1148
254 1148
256 1148
258 1148
260 1148
262 1148
264 1148
266 1148
268 1148
270 1148
272 1148
274 1148
276 1148
278 1148
280 1148
282 1148
284 1148
286 1148
288 1148
290 1148
292 1148
294 1148
296 1148
298 1148
300 1148
302 1148
304 1148
306 1148
308 1148
310 1148
312 1148
314 1148
316 1148
318 1148
320 1148
322 1148
324 1148
326 1148
328 1148
330 1148
332 1148
334 1148
336 1148
338 1148
340 1148
342 1148
344 1148
346 1148
348 1148
350 1148
352 1148
354 1148
356 1148
358 1148
360 1148
362 1148
364 1148
366 1148
368 1148
370 1148
372 1148
374 1148
406 1148
408 1148
410 1148
412 1148
414 1148
416 1148
418 1148
420 1148
422 1148
120 1150
122 1150
124 1150
126 1150
128 1150
130 1150
132 1150
134 1150
136 1150
138 1150
156 1150
158 1150
160 1150
162 1150
164 1150
166 1150
168 1150
170 1150
172 1150
174 1150
176 1150
178 1150
180 1150
182 1150
184 1150
186 1150
188 1150
190 1150
192 1150
194 1150
196 1150
198 1150
200 1150
202 1150
2

274 1164
276 1164
278 1164
280 1164
282 1164
284 1164
286 1164
288 1164
290 1164
292 1164
294 1164
296 1164
298 1164
300 1164
302 1164
304 1164
306 1164
308 1164
310 1164
312 1164
314 1164
316 1164
318 1164
320 1164
322 1164
324 1164
326 1164
328 1164
330 1164
332 1164
334 1164
336 1164
338 1164
340 1164
342 1164
344 1164
346 1164
348 1164
350 1164
380 1164
382 1164
384 1164
406 1164
144 1166
146 1166
148 1166
176 1166
178 1166
180 1166
182 1166
184 1166
186 1166
188 1166
190 1166
192 1166
194 1166
196 1166
198 1166
200 1166
202 1166
204 1166
206 1166
208 1166
210 1166
212 1166
214 1166
216 1166
218 1166
220 1166
222 1166
224 1166
226 1166
228 1166
230 1166
232 1166
234 1166
236 1166
238 1166
240 1166
242 1166
244 1166
246 1166
248 1166
250 1166
252 1166
254 1166
256 1166
258 1166
260 1166
262 1166
264 1166
266 1166
268 1166
270 1166
272 1166
274 1166
276 1166
278 1166
280 1166
282 1166
284 1166
286 1166
288 1166
290 1166
292 1166
294 1166
296 1166
298 1166
300 1166
302 1166
304 1166
3

338 1186
340 1186
342 1186
358 1186
360 1186
362 1186
364 1186
396 1186
398 1186
400 1186
168 1188
170 1188
244 1188
246 1188
248 1188
250 1188
252 1188
254 1188
256 1188
258 1188
260 1188
262 1188
264 1188
310 1188
322 1188
330 1188
332 1188
334 1188
336 1188
338 1188
346 1188
348 1188
350 1188
352 1188
354 1188
356 1188
364 1188
386 1188
388 1188
390 1188
392 1188
394 1188
396 1188
398 1188
400 1188
168 1190
170 1190
172 1190
174 1190
176 1190
178 1190
180 1190
182 1190
184 1190
186 1190
188 1190
190 1190
192 1190
194 1190
196 1190
198 1190
200 1190
202 1190
204 1190
206 1190
208 1190
210 1190
212 1190
214 1190
216 1190
218 1190
220 1190
250 1190
252 1190
302 1190
328 1190
330 1190
332 1190
334 1190
336 1190
350 1190
356 1190
386 1190
388 1190
390 1190
392 1190
394 1190
396 1190
398 1190
168 1192
170 1192
172 1192
174 1192
176 1192
178 1192
180 1192
182 1192
184 1192
186 1192
188 1192
190 1192
192 1192
194 1192
196 1192
198 1192
200 1192
202 1192
204 1192
206 1192
208 1192
210 1192
2